In [1]:
import mysql.connector

import datetime

import csv

import pandas as pd

from sqlalchemy import create_engine

import yfinance as yf

### connect to google cloud sql

In [2]:
import pymysql
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             password='PApOEaPbJNOM62p1',
                             db='stockdb'
                            ,port=1234)

In [3]:
cur = connection.cursor()


### get the period of time we are interested in 

In [8]:
now=datetime.date.today()
start = datetime.datetime(2020, 1, 1)
end = now

### read in the stock symbols from 500.csv 

In [9]:
def companydata(file):
    companyfull=list()
    with open(file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        #next(reader)
        for row in reader:
            companyfull.append(row)
    company=pd.DataFrame(companyfull)
    return company


In [10]:
company=companydata('500.csv')
companysymbol=company["Symbol"].tolist()
company1=company.drop(columns=['Name'])
Symbols=company['Symbol'].tolist()


In [41]:
company["new_name"] = company['Name'].str.replace("[']",'')


In [42]:
company


,Symbol,Name,Sector,new_name
0,MMM,3M Company,Industrials,3M Company
1,AOS,A.O. Smith Corp,Industrials,A.O. Smith Corp
2,ABT,Abbott Laboratories,Health Care,Abbott Laboratories
3,ABBV,AbbVie Inc.,Health Care,AbbVie Inc.
4,ABMD,ABIOMED Inc,Health Care,ABIOMED Inc
...,...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary,Yum! Brands Inc
501,ZBRA,Zebra Technologies,Information Technology,Zebra Technologies
502,ZBH,Zimmer Biomet Holdings,Health Care,Zimmer Biomet Holdings
503,ZION,Zions Bancorp,Financials,Zions Bancorp


In [52]:
sqlcode1=('Select * From CompanyInformation')
cur.execute(sqlcode1)
test1 = cur.fetchone()
test1


('A', 'Agilent Technologies Inc', 'Health Care')

### insert data into google cloud sql, create table companyinformation

In [45]:
for row in company.iterrows():

    idx = row[0]
    symbol = row[1][0]
    name = row[1][3]
    sector = row[1][2]

    try: 
        cur.execute(f"INSERT INTO CompanyInformation (symbol,name,sector) VALUES ('{symbol}','{name}', '{sector}');")

    except Exception as e:
        print(e)
        continue
        

In [53]:
sqlcode1=('Select * From CompanyInformation Where Sector="Industrials"')
cur.execute(sqlcode1)
Industrials = cur.fetchone()
Industrials


('AAL', 'American Airlines Group', 'Industrials')

### create one table for each stock


In [12]:
for i in Symbols:
    symbol=yf.Ticker(i)
    df1 = symbol.history(start = start,end=end)
    df1['DATE']= pd.to_datetime(df1.index)
    try:
        cur.execute("CREATE TABLE "+i+"(date date,open varchar(250),high varchar(250),low varchar(250),close varchar(250),volume varchar(250),dividends varchar(250))")
        print('creating'+i)
        for row in df1.iterrows():
            date=row[1][7]
            open1 = row[1][0]
            high = row[1][1]
            low = row[1][2]
            close = row[1][3]
            volume = row[1][4]
            dividends=row[1][5]
            #cur.execute(f"INSERT INTO MMM (date,open,high,low,close,volume,dividends) VALUES ('{date}','{open1}','{high}','{low}','{close}','{volume}','{dividends}');")
            #f"(date,open,high,low,close,volume,dividends) VALUES ('{date}','{open1}','{high}','{low}','{close}','{volume}','{dividends}');"
            cur.execute(f"INSERT INTO "+ i +"(date,open,high,low,close,volume,dividends) VALUES "+f"('{date}','{open1}','{high}','{low}','{close}','{volume}','{dividends}');")
            
    except Exception as e:
        print(e)
        continue
    

(1050, "Table 'MMM' already exists")
(1050, "Table 'AOS' already exists")
(1050, "Table 'ABT' already exists")
(1050, "Table 'ABBV' already exists")
(1050, "Table 'ABMD' already exists")
(1050, "Table 'ACN' already exists")
(1050, "Table 'ATVI' already exists")
(1050, "Table 'ADBE' already exists")
(1050, "Table 'AAP' already exists")
(1050, "Table 'AMD' already exists")
(1050, "Table 'AES' already exists")
(1050, "Table 'AFL' already exists")
(1050, "Table 'A' already exists")
(1050, "Table 'APD' already exists")
(1050, "Table 'AKAM' already exists")
(1050, "Table 'ALK' already exists")
(1050, "Table 'ALB' already exists")
(1050, "Table 'ARE' already exists")
(1050, "Table 'ALXN' already exists")
(1050, "Table 'ALGN' already exists")
(1050, "Table 'ALLE' already exists")
(1050, "Table 'LNT' already exists")
(1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'ALL(date date,open varchar(250),high 

creatingMCO
creatingMS
creatingMSI
creatingMSCI
creatingMYL
creatingNDAQ
creatingNOV
creatingNTAP
creatingNFLX
creatingNWL
creatingNEM
creatingNWSA
creatingNWS
creatingNEE
creatingNLSN
creatingNKE
creatingNI
creatingNBL
creatingNSC
creatingNTRS
creatingNOC
creatingNLOK
creatingNCLH
creatingNRG
creatingNUE
creatingNVDA
creatingNVR
creatingORLY
creatingOXY
creatingODFL
creatingOMC
creatingOKE
creatingORCL
creatingOTIS
creatingPCAR
creatingPKG
creatingPH
creatingPAYX
creatingPAYC
creatingPYPL
creatingPNR
creatingPBCT
creatingPEP
creatingPKI
creatingPRGO
creatingPFE
creatingPM
creatingPSX
creatingPNW
creatingPXD
creatingPNC
creatingPPG
creatingPPL
creatingPFG
creatingPG
creatingPGR
creatingPLD
creatingPRU
creatingPEG
creatingPSA
creatingPHM
creatingPVH
creatingQRVO
creatingQCOM
creatingPWR
creatingDGX
creatingRL
creatingRJF
creatingRTX
creatingO
creatingREG
creatingREGN
creatingRF
creatingRSG
creatingRMD
creatingRHI
creatingROK
creatingROL
creatingROP
creatingROST
creatingRCL
creatingSPGI


In [239]:
sqlcode1=('Select * From A')
cur.execute(sqlcode1)
Industrials = cur.fetchone()
Industrials


(datetime.date(2020, 1, 2), '85.35', '85.8', '84.65', '85.4', '1410500', '0.0')